<a href="https://colab.research.google.com/github/2019ht12169/Thesis/blob/main/parse_and_clean_data_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!pip install better_profanity
import json
import gzip
from bs4 import BeautifulSoup
import re
from better_profanity import profanity
import pandas as pd 


     |████████████████████████████████| 46 kB 1.7 MB/s 


In [1]:
from google.colab import files
upload=files.upload()

In [2]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [3]:
chat_words_str = """
AFAIK=As Far As I Know
ASAP=As Soon As Possible
ATM=At The Moment
A3=Anytime, Anywhere, Anyplace
BRT=Be Right There
BTW=By The Way
B4=Before
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FWIW=For What It's Worth
FYI=For Your Information
GAL=Get A Life
GG=Good Game
GN=Good Night
GMTA=Great Minds Think Alike
GR8=Great!
G9=Genius
IC=I See
ILU=ILU: I Love You
IMHO=In My Honest/Humble Opinion
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
KISS=Keep It Simple, Stupid
LDR=Long Distance Relationship
LMAO=Laugh My Ass Off
LOL=Laughing Out Loud
LTNS=Long Time No See
L8R=Later
MTE=My Thoughts Exactly
M8=Mate
OIC=Oh I See
PITA=Pain In The Ass
PRT=Party
PRW=Parents Are Watching
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My Ass Off
SK8=Skate
STATS=Your sex and age
ASL=Age, Sex, Location
THX=Thank You
U=You
U2=You Too
WB=Welcome Back
WTF=What The Fuck
WTG=Way To Go!
WUF=Where Are You From?
W8=Wait...
7K=Sick:-D Laugher
"""
chat_words_map_dict = {}
chat_words_list = []
for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)

def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [4]:
EMOTICONS = {
    u":‑\)":"Happy face",
    u":\)":"Happy face",
    u":-\]":"Happy face",
    u":\]":"Happy face",
    u":-3":"Happy face",
    u":3":"Happy face",
    u":->":"Happy face",
    u":>":"Happy face",
    u"8-\)":"Happy face",
    u":o\)":"Happy face",
    u":-\}":"Happy face",
    u":\}":"Happy face",
    u":-\)":"Happy face",
    u":c\)":"Happy face",
    u":\^\)":"Happy face",
    u"=\]":"Happy face",
    u"=\)":"Happy face",
    u":‑D":"Laughing",
    u":D":"Laughing",
    u"8‑D":"Laughing",
    u"8D":"Laughing",
    u"X‑D":"Laughing",
    u"XD":"Laughing",
    u"=D":"Laughing",
    u"=3":"Laughing",
    u"B\^D":"Laughing",
    u":-\)\)":"Very happy",
    u":‑\(":"angry",
    u":-\(":"angry",
    u":\(":"angry",
    u":‑c":"angry",
    u":c":"angry",
    u":‑<":"angry",
    u":<":"angry",
    u":‑\[":"angry",
    u":\[":"angry",
    u":-\|\|":"angry",
    u">:\[":"angry",
    u":\{":"angry",
    u":@":"angry",
    u">:\(":"angry",
    u":'‑\(":"Crying",
    u":'\(":"Crying",
    u":'‑\)":"Tears of happiness",
    u":'\)":"Tears of happiness",
    u"D‑':":"Horror",
    u"D:<":"Disgust",
    u"D:":"Sadness",
    u"D8":"Great dismay",
    u"D;":"Great dismay",
    u"D=":"Great dismay",
    u"DX":"Great dismay",
    u":‑O":"Surprise",
    u":O":"Surprise",
    u":‑o":"Surprise",
    u":o":"Surprise",
    u":-0":"Shock",
    u"8‑0":"Yawn",
    u">:O":"Yawn",
    u":-\*":"Kiss",
    u":\*":"Kiss",
    u":X":"Kiss",
    u";‑\)":"smirk",
    u";\)":"smirk",
    u"\*-\)":"smirk",
    u"\*\)":"smirk",
    u";‑\]":"smirk",
    u";\]":"smirk",
    u";\^\)":"smirk",
    u":‑,":"smirk",
    u";D":"smirk",
    u":‑P":"Tongue sticking out",
    u":P":"Tongue sticking out",
    u"X‑P":"Tongue sticking out",
    u"XP":"Tongue sticking out",
    u":‑Þ":"Tongue sticking out",
    u":Þ":"Tongue sticking out",
    u":b":"Tongue sticking out",
    u"d:":"Tongue sticking out",
    u"=p":"Tongue sticking out",
    u">:P":"Tongue sticking out",
    u":‑/":"Skeptical",
    u":/":"Skeptical",
    u":-[.]":"Skeptical",
    u">:[(\\\)]":"Skeptical",
    u">:/":"Skeptical",
    u":[(\\\)]":"Skeptical",
    u"=/":"Skeptical",
    u"=[(\\\)]":"Skeptical",
    u":L":"Skeptical",
    u"=L":"Skeptical",
    u":S":"Skeptical",
    u":‑\|":"Straight face",
    u":\|":"Straight face",
    u":$":"Embarrassed",
    u":‑x":"Sealed lips",
    u":x":"Sealed lips",
    u":‑#":"Sealed lips",
    u":#":"Sealed lips",
    u":‑&":"Sealed lips",
    u":&":"Sealed lips",
    u"O:‑\)":"innocent",
    u"O:\)":"innocent",
    u"0:‑3":"innocent",
    u"0:3":"innocent",
    u"0:‑\)":"innocent",
    u"0:\)":"innocent",
    u":‑b":"Tongue sticking out",
    u"0;\^\)":"innocent",
    u">:‑\)":"devilish",
    u">:\)":"devilish",
    u"\}:‑\)":"devilish",
    u"\}:\)":"devilish",
    u"3:‑\)":"devilish",
    u"3:\)":"devilish",
    u">;\)":"devilish",
    u"\|;‑\)":"Cool",
    u"\|‑O":"Bored",
    u":‑J":"Tongue-in-cheek",
    u"#‑\)":"Party all night",
    u"%‑\)":"confused",
    u"%\)":"confused",
    u":-###..":"Being sick",
    u":###..":"Being sick",
    u"<:‑\|":"Dump",
    u"\(>_<\)":"Troubled",
    u"\(>_<\)>":"Troubled",
    u"\(';'\)":"Baby",
    u"\(\^\^>``":"Embarrassed",
    u"\(\^_\^;\)":"Embarrassed",
    u"\(-_-;\)":"Embarrassed",
    u"\(~_~;\) \(・\.・;\)":"Embarrassed",
    u"\(-_-\)zzz":"Sleeping",
    u"\(\^_-\)":"Wink",
    u"\(\(\+_\+\)\)":"Confused",
    u"\(\+o\+\)":"Confused",
    u"\(o\|o\)":"Ultraman",
    u"\^_\^":"Joyful",
    u"\(\^_\^\)/":"Joyful",
    u"\(\^O\^\)／":"Joyful",
    u"\(\^o\^\)／":"Joyful",
    u"\(__\)":"sign of respect",
    u"_\(\._\.\)_":"sign of respect",
    u"<\(_ _\)>":"sign of respect",
    u"<m\(__\)m>":"sign of respect",
    u"m\(__\)m":"sign of respect",
    u"m\(_ _\)m":"sign of respect",
    u"\('_'\)":"Crying",
    u"\(/_;\)":"Crying",
    u"\(T_T\) \(;_;\)":"Crying",
    u"\(;_;":"Crying",
    u"\(;_:\)":"Crying",
    u"\(;O;\)":"Crying",
    u"\(:_;\)":"Crying",
    u"\(ToT\)":"Crying",
    u";_;":"Crying",
    u";-;":"Crying",
    u";n;":"Crying",
    u";;":"Crying",
    u"Q\.Q":"Crying",
    u"T\.T":"Crying",
    u"QQ":"Crying",
    u"Q_Q":"Crying",
    u"\(-\.-\)":"Shame",
    u"\(-_-\)":"Shame",
    u"\(一一\)":"Shame",
    u"\(；一_一\)":"Shame",
    u"\(=_=\)":"Tired",
    u"\(=\^\·\^=\)":"cat",
    u"\(=\^\·\·\^=\)":"cat",
    u"=_\^=	":"cat",
    u"\(\.\.\)":"Looking down",
    u"\(\._\.\)":"Looking down",
    u"\^m\^":"Giggling",
    u"\(\・\・?":"Confusion",
    u"\(?_?\)":"Confusion",
    u">\^_\^<":"Normal Laugh",
    u"<\^!\^>":"Normal Laugh",
    u"\^/\^":"Normal Laugh",
    u"\（\*\^_\^\*）" :"Normal Laugh",
    u"\(\^<\^\) \(\^\.\^\)":"Normal Laugh",
    u"\(^\^\)":"Normal Laugh",
    u"\(\^\.\^\)":"Normal Laugh",
    u"\(\^_\^\.\)":"Normal Laugh",
    u"\(\^_\^\)":"Normal Laugh",
    u"\(\^\^\)":"Normal Laugh",
    u"\(\^J\^\)":"Normal Laugh",
    u"\(\*\^\.\^\*\)":"Normal Laugh",
    u"\(\^—\^\）":"Normal Laugh",
    u"\(#\^\.\^#\)":"Normal Laugh",
    u"\（\^—\^\）":"Waving",
    u"\(;_;\)/~~~":"Waving",
    u"\(\^\.\^\)/~~~":"Waving",
    u"\(-_-\)/~~~ \($\·\·\)/~~~":"Waving",
    u"\(T_T\)/~~~":"Waving",
    u"\(ToT\)/~~~":"Waving",
    u"\(\*\^0\^\*\)":"Excited",
    u"\(\*_\*\)":"Amazed",
    u"\(\*_\*;":"Amazed",
    u"\(\+_\+\) \(@_@\)":"Amazed",
    u"\(\*\^\^\)v":"Cheerful",
    u"\(\^_\^\)v":"Cheerful",
    u"\(\(d[-_-]b\)\)":"Listening to music",
    u'\(-"-\)':"Worried",
    u"\(ーー;\)":"Worried",
    u"\(\^0_0\^\)":"Eyeglasses",
    u"\(\＾ｖ\＾\)":"Happy",
    u"\(\＾ｕ\＾\)":"Happy",
    u"\(\^\)o\(\^\)":"Happy",
    u"\(\^O\^\)":"Happy",
    u"\(\^o\^\)":"Happy",
    u"\)\^o\^\(":"Happy",
    u":O o_O":"Surprised",
    u"o_0":"Surprised",
    u"o\.O":"Surpised",
    u"\(o\.o\)":"Surprised",
    u"oO":"Surprised",
    u"\(\*￣m￣\)":"Dissatisfied",
    u"\(‘A`\)":"Deflated"
}
def convert_emoticons(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', ' '.join(EMOTICONS[emot].replace(",","").split()), text)
    return text

In [5]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords_default = stopwords.words('english')
stopwords_toBeRemoved=['no', 'nor', 'not','but','don',"don't","aren't",'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't",'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
for word in stopwords_toBeRemoved:
  stopwords_default.remove(word)
STOPWORDS = set(stopwords_default)
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
def preprocessText(reviewText):
      # remove url's
      reviewText = remove_urls(reviewText)
      # remove html tags
      soup = BeautifulSoup(reviewText)
      reviewText = soup.get_text()
      # remove new line characters
      #reviewText = reviewText.replace('\n',' ')
      # convert acronyms/chat terminology to full text
      reviewText = chat_words_conversion(reviewText)
      #convert emoticons to text
      reviewText = convert_emoticons(reviewText)
      # remove stop words
      reviewText = remove_stopwords(reviewText)
      # remove punctuation 
      #reviewText = re.sub(r'[^\w\s]', '', reviewText)
      reviewText = reviewText.replace('"','')
      # do censoring
      #reviewText = profanity.censor(reviewText)
      #just comma to avoid parsing conflicts
      reviewText = re.sub(',', '', reviewText)
      #remove astricks to nullify bad words
      #reviewText = reviewText.replace('*','')
      # convert all text to lower case
      reviewText = reviewText.lower()
      return reviewText

In [11]:
#download data file
tsv_file='amazon_reviews_Electronics.tsv'
csv_table=pd.read_table(tsv_file,sep='\t',error_bad_lines=False)
# convert data file into csv format
csv_table.to_csv('reviews_Electronics.csv',index=False,header=True)


b'Skipping line 9076: expected 15 fields, saw 22\nSkipping line 19256: expected 15 fields, saw 22\nSkipping line 24313: expected 15 fields, saw 22\nSkipping line 47211: expected 15 fields, saw 22\nSkipping line 54295: expected 15 fields, saw 22\nSkipping line 56641: expected 15 fields, saw 22\n'


In [12]:
#making data frame out of the csv version of data file
data = pd.read_csv('reviews_Electronics.csv')
# drop unneeded columns and print it
df = data.drop(['marketplace','customer_id','review_id','product_id',
       'product_parent','product_title', 'product_category','helpful_votes', 'total_votes', 'vine', 'verified_purchase'], axis='columns')
df.head()

,star_rating,review_headline,review_body,review_date
0,5,Five Stars,As described.,2015-08-31
1,5,It works as advertising.,It works as advertising.,2015-08-31
2,5,Five Stars,Works pissa,2015-08-31
3,1,One Star,Did not work at all.,2015-08-31
4,5,Overall pleased with the item,Works well. Bass is somewhat lacking but is pr...,2015-08-31


In [13]:
#create new attributes as per need 
df.loc[df['star_rating'] < 4, 'label'] = '0' 
df.loc[df['star_rating'] >= 4, 'label'] = '1' 
df=df.assign(text=lambda x: x['review_headline']+' '+x['review_body'])

In [14]:
df_map_result = pd.DataFrame({'label': df['label'],
    'text': df['text'],
    'review_date': df['review_date']})

df_map_result.to_csv("dataFile_reviews_Electronics.csv",index=False,header=True)

In [16]:
import csv
file1 = open("dataFile_latest_1.csv","w")
count=0
with open('dataFile_reviews_Electronics.csv', 'r') as file:
    my_reader = csv.reader(file, delimiter=',')
    for row in my_reader:
      count = count + 1
      file1.write('"{}"'.format(row[0]))
      file1.write(",")
      file1.write('"{}"'.format(preprocessText(row[1])))
      file1.write(",")
      file1.write('"{}"'.format(row[2]))
      file1.write("\n")
    print(count)

57595
